<a href="https://colab.research.google.com/github/akash166d/sparkNLP/blob/master/NLP_sentiment_eg_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Package/Lib

This is required to ensure the ubuntu related dependencies are up to date

In [ ]:
! sudo apt-get update --fix-missing
! sudo apt-get upgrade --fix-missing

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-ba

In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5.4

! pip install --user -U nltk

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
     |████████████████████████████████| 215.7MB 52kB/s 
     |████████████████████████████████| 204kB 43.1MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434674 sha256=dfa509c009c6abd1ac4a464b7fe76d736d43b2813f043300555749c71156befc
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.4
Apache Spark version;  2.4.4


# Mount Drive and read data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


Some of the sparkNLP annotators were having trouble reading spaces in path, hence used symbolic link to replace the path

In [ ]:
! ln -s "/content/drive/My Drive" "/content/MyDrive"

In [ ]:

song_data = spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/SparkNLP/song_2k.csv")


In [ ]:
from pyspark.sql import functions as F
song_data = song_data.select(['Key', 'Lyric', 'Genre']).withColumnRenamed('Lyric','text')

# train, test = trainDataset.randomSplit(weights=[0.5, 0.5], seed=123)
song_data = song_data.limit(999)
print(song_data.count())

song_data = song_data.filter(song_data.text != '')
print(song_data.count())
song_data = song_data.filter((song_data.Genre == 'Rock') | (song_data.Genre == 'Hip Hop') | (song_data.Genre == 'Pop')  )
# song_data = song_data.filter(song_data.Genre != '')
print(song_data.count())
# song_data = song_data.filter(song_data.text != ' ')
# print(song_data.count())
# song_data = song_data.filter(song_data.Genre != ' ')
# print(song_data.count())
song_data.na.drop(subset=["text"])
print(song_data.count())
song_data.show()

999
999
915
915
+--------------------+--------------------+-------+
|                 Key|                text|  Genre|
+--------------------+--------------------+-------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|
|jamiroquai_Rock D...|And it's coming a...|    Pop|
|10000 maniacs_The...|These are. These ...|   Rock|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|
|black veil brides...|Have we begun to ...|   Rock|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|
|10000 maniacs_Acr...|Well they left th...|   Rock|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|
|twista_Back 2 School|[Tung Twista]. ba...|Hip Hop|
|10000 maniacs_Ant...|For whom do the b...|   Rock|
|10000 maniacs_All...|She walks alone o...|   Rock|
|10000 maniacs_Bac...|Jenny. Jenny you ...|   Rock|
|cyndi lauper_True...|You with the sad ...|    P

In [ ]:
print(type(song_data))

<class 'pyspark.sql.dataframe.DataFrame'>


# Package Import

In [ ]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from pyspark.sql import functions as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Vocab Reduction

Basic Data cleaning for NLP

In [ ]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)

# token_assembler = TokenAssembler().setInputCols(["cleanTokens"]).setOutputCol("assembled")

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True)#.setValueSplitSymbol(" ")

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
pipeline_bow = Pipeline().setStages([
    assembler, tokenizer, spell_chk , 
    lemmatizer, normalizer,stop_words_cleaner,finisher
])
model_trans_tfIdf = pipeline_bow.fit(song_data)
model_trans_tfIdf =  model_trans_tfIdf.transform(song_data)
model_trans_tfIdf.show()

+--------------------+--------------------+-------+--------------------+
|                 Key|                text|  Genre|         cleanTokens|
+--------------------+--------------------+-------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[grudge, slowly, ...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[donut, talk, lis...|
|black veil brides...|Have we begun to ...|   Rock|[begin, drift, aw...|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|[aint, need, worr...|
|10000 maniacs_Acr...|Well they left th...|   Rock|[well, leave, mor...|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|[music, dennis, d...|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|

In [ ]:
#convert list to sentence
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import *
@udf( StringType())
def get_sentence(word_in):
    return " ".join(str(item) for item in word_in)

# udf_to_doc = udf(get_sentence, StringType())

df_cleanToken = model_trans_tfIdf.select('Key','Genre', get_sentence(F.col('cleanTokens')).alias('text'))
# bow_top_embd = bow_top_embd.filter((F.col('doc') != '') | F.col('doc') != ' ')
df_cleanToken.na.drop(subset=["text"])
from pyspark.sql.functions import trim
df_cleanToken = df_cleanToken.withColumn("text", trim(df_cleanToken.text))
# bow_top_embd.filter(!(F.col('doc') == ' ')).show()
df_cleanToken.show()

+--------------------+-------+--------------------+
|                 Key|  Genre|                text|
+--------------------+-------+--------------------+
|10000 maniacs_Mor...|   Rock|could feel time w...|
|10000 maniacs_Bec...|   Rock|take baby hold cl...|
|jamiroquai_Rock D...|    Pop|come baby yeah co...|
|10000 maniacs_The...|   Rock|day youll remembe...|
|10000 maniacs_Eve...|   Rock|grudge slowly wet...|
|10000 maniacs_Don...|   Rock|donut talk listen...|
|black veil brides...|   Rock|begin drift away ...|
|lynyrd skynyrd_I ...|   Rock|aint need worry a...|
|10000 maniacs_Acr...|   Rock|well leave mornin...|
|10000 maniacs_Pla...|   Rock|music dennis drew...|
|10000 maniacs_Rai...|   Rock|bend kneeive look...|
|twista_Back 2 School|Hip Hop|tung twista back ...|
|10000 maniacs_Ant...|   Rock|bell toll sentenc...|
|10000 maniacs_All...|   Rock|walk alone brick ...|
|10000 maniacs_Bac...|   Rock|jenny jenny dont ...|
|cyndi lauper_True...|    Pop|sad eye donut dis...|
|10000 mania

# Unsupervised Sentiment Analysis

## Analyze_sentiment

In [ ]:
pipeline = PretrainedPipeline("analyze_sentiment", lang="en")
# mod_sent = pipeline.fit(model_trans_tfIdf)
sent_df = pipeline.transform(df_cleanToken)
sent_df = sent_df.select('text','sentiment.result' ,'sentiment.metadata' )
sent_df.show(truncate = True)


analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]
+--------------------+----------+--------------------+
|                text|    result|            metadata|
+--------------------+----------+--------------------+
|could feel time w...|[positive]|[[confidence -> 0...|
|take baby hold cl...|[positive]|[[confidence -> 0...|
|come baby yeah co...|[positive]|[[confidence -> 0...|
|day youll remembe...|[positive]|[[confidence -> 0...|
|grudge slowly wet...|[positive]|[[confidence -> 0...|
|donut talk listen...|[negative]|[[confidence -> 0...|
|begin drift away ...|[positive]|[[confidence -> 0...|
|aint need worry a...|[positive]|[[confidence -> 0...|
|well leave mornin...|[positive]|[[confidence -> 0...|
|music dennis drew...|[positive]|[[confidence -> 0...|
|bend kneeive look...|[positive]|[[confidence -> 0...|
|tung twista back ...|[positive]|[[confidence -> 0...|
|bell toll sentenc...|[positive]|[[confidence -> 0...|
|walk alone brick ...|[

Save sentiment result and corresponding confidence

1. find result and confidence column
2. find top rows based on confidence and find top words for them using TF-IDF rank

In [ ]:
#convert list to sentence
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import *

@udf(StringType())
def get_result(sent_result):
    result = sent_result[0]
    return str(result)

@udf(FloatType())
def get_conf(sent_conf):
    result = sent_conf[0]
    conf = result.get('confidence')
    return float(conf)

df_sent_result = sent_df.select('text',get_result(F.col('result')).alias('result'),get_conf('metadata').alias('confidence') ) #
df_sent_result.na.drop(subset=["confidence"])
df_sent_result.show()
df_sent_result.count()

+--------------------+--------+----------+
|                text|  result|confidence|
+--------------------+--------+----------+
|could feel time w...|positive|    0.6221|
|take baby hold cl...|positive|    0.8311|
|come baby yeah co...|positive|    0.8025|
|day youll remembe...|positive|    0.5757|
|grudge slowly wet...|positive|    0.9414|
|donut talk listen...|negative|    0.4813|
|begin drift away ...|positive|    0.7405|
|aint need worry a...|positive|    0.6867|
|well leave mornin...|positive|    0.9522|
|music dennis drew...|positive|    0.7857|
|bend kneeive look...|positive|    0.8943|
|tung twista back ...|positive|    0.7895|
|bell toll sentenc...|positive|    0.6725|
|walk alone brick ...|positive|    0.7489|
|jenny jenny dont ...|positive|    0.5447|
|sad eye donut dis...|positive|    0.8772|
|look away western...|positive|    0.4822|
|make could ever m...|positive|    0.8102|
|im keeper heart k...|negative|    0.8324|
|leg maddox kitche...|positive|    0.5447|
+----------

915

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(df_sent_result['result']).orderBy(df_sent_result['confidence'].desc())

df_sent_result = df_sent_result.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 50)
df_sent_result.show()

+--------------------+--------+----------+----+
|                text|  result|confidence|rank|
+--------------------+--------+----------+----+
|tell lie deceitfu...|positive|     0.984|   1|
|deep solace lie u...|positive|    0.9771|   2|
|another awesome s...|positive|    0.9703|   3|
|come everything o...|positive|    0.9546|   4|
|love break apart ...|positive|    0.9539|   5|
|well leave mornin...|positive|    0.9522|   6|
|well leave mornin...|positive|    0.9522|   6|
|grudge slowly wet...|positive|    0.9414|   8|
|grudge slowly wet...|positive|    0.9414|   8|
|ben sweller verse...|positive|    0.9388|  10|
|letâs throw twili...|positive|    0.9285|  11|
|wasnt restless fi...|positive|    0.9281|  12|
|trocha fire go si...|positive|    0.9239|  13|
|lucille please co...|positive|    0.9217|  14|
|never know day li...|positive|    0.9107|  15|
|fear fall hard li...|positive|    0.9096|  16|
|fear fall hard li...|positive|    0.9096|  16|
|dig u picture u e...|positive|    0.908

### Top Positive word

In [ ]:
df_positive = df_sent_result.filter(df_sent_result.result == 'positive')
df_positive.show()

+--------------------+--------+----------+----+
|                text|  result|confidence|rank|
+--------------------+--------+----------+----+
|tell lie deceitfu...|positive|     0.984|   1|
|deep solace lie u...|positive|    0.9771|   2|
|another awesome s...|positive|    0.9703|   3|
|come everything o...|positive|    0.9546|   4|
|love break apart ...|positive|    0.9539|   5|
|well leave mornin...|positive|    0.9522|   6|
|well leave mornin...|positive|    0.9522|   6|
|grudge slowly wet...|positive|    0.9414|   8|
|grudge slowly wet...|positive|    0.9414|   8|
|ben dweller verse...|positive|    0.9388|  10|
|letâs throw twili...|positive|    0.9285|  11|
|wasnt restless fi...|positive|    0.9281|  12|
|trochi fire go si...|positive|    0.9239|  13|
|lucille please co...|positive|    0.9217|  14|
|never know day li...|positive|    0.9107|  15|
|fear fall hard li...|positive|    0.9096|  16|
|fear fall hard li...|positive|    0.9096|  16|
|dig u picture u e...|positive|    0.908

In [ ]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')
tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')
finisher = Finisher().setInputCols(['token']).setOutputCols(['text']).setOutputAsArray(True)

pipeline_bow = Pipeline().setStages([
    assembler, tokenizer, finisher
])
model_trans_tfIdf = pipeline_bow.fit(df_positive)
top_positive =  model_trans_tfIdf.transform(df_positive)
top_positive.show()

+--------------------+--------+----------+----+
|                text|  result|confidence|rank|
+--------------------+--------+----------+----+
|[tell, lie, decei...|positive|     0.984|   1|
|[deep, solace, li...|positive|    0.9771|   2|
|[another, awesome...|positive|    0.9703|   3|
|[come, everything...|positive|    0.9546|   4|
|[love, break, apa...|positive|    0.9539|   5|
|[well, leave, mor...|positive|    0.9522|   6|
|[well, leave, mor...|positive|    0.9522|   6|
|[grudge, slowly, ...|positive|    0.9414|   8|
|[grudge, slowly, ...|positive|    0.9414|   8|
|[ben, dweller, ve...|positive|    0.9388|  10|
|[letâs, throw, tw...|positive|    0.9285|  11|
|[dasnt, restless,...|positive|    0.9281|  12|
|[trucha, fire, go...|positive|    0.9239|  13|
|[lucille, please,...|positive|    0.9217|  14|
|[never, know, day...|positive|    0.9107|  15|
|[fear, fall, hard...|positive|    0.9096|  16|
|[fear, fall, hard...|positive|    0.9096|  16|
|[dig, u, picture,...|positive|    0.907

In [ ]:
from pyspark.ml.feature import CountVectorizer
tfizer = CountVectorizer(inputCol='text', outputCol='tf_features')
tf_model = tfizer.fit(top_positive)
tf_result = tf_model.transform(top_positive)
tf_result.show()

+--------------------+--------+----------+----+--------------------+
|                text|  result|confidence|rank|         tf_features|
+--------------------+--------+----------+----+--------------------+
|[tell, lie, decei...|positive|     0.984|   1|(1124,[0,32,37,38...|
|[deep, solace, li...|positive|    0.9771|   2|(1124,[0,16,37,46...|
|[another, awesome...|positive|    0.9703|   3|(1124,[7,11,23,27...|
|[come, everything...|positive|    0.9546|   4|(1124,[0,5,6,7,11...|
|[love, break, apa...|positive|    0.9539|   5|(1124,[0,3,55,57,...|
|[well, leave, mor...|positive|    0.9522|   6|(1124,[0,7,15,23,...|
|[well, leave, mor...|positive|    0.9522|   6|(1124,[0,7,15,23,...|
|[grudge, slowly, ...|positive|    0.9414|   8|(1124,[1,6,28,65,...|
|[grudge, slowly, ...|positive|    0.9414|   8|(1124,[1,6,28,65,...|
|[ben, dweller, ve...|positive|    0.9388|  10|(1124,[0,10,11,17...|
|[letâs, throw, tw...|positive|    0.9285|  11|(1124,[1,7,10,34,...|
|[hasnt, restless,...|positive|   

In [ ]:
from pyspark.ml.feature import IDF
idfizer = IDF(inputCol='tf_features', 
              outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)
tfidf_result.show()

+--------------------+--------+----------+----+--------------------+--------------------+
|                text|  result|confidence|rank|         tf_features|     tf_idf_features|
+--------------------+--------+----------+----+--------------------+--------------------+
|[tell, lie, decei...|positive|     0.984|   1|(1124,[0,32,37,38...|(1124,[0,32,37,38...|
|[deep, solace, li...|positive|    0.9771|   2|(1124,[0,16,37,46...|(1124,[0,16,37,46...|
|[another, awesome...|positive|    0.9703|   3|(1124,[7,11,23,27...|(1124,[7,11,23,27...|
|[come, everything...|positive|    0.9546|   4|(1124,[0,5,6,7,11...|(1124,[0,5,6,7,11...|
|[love, break, apa...|positive|    0.9539|   5|(1124,[0,3,55,57,...|(1124,[0,3,55,57,...|
|[well, leave, mor...|positive|    0.9522|   6|(1124,[0,7,15,23,...|(1124,[0,7,15,23,...|
|[well, leave, mor...|positive|    0.9522|   6|(1124,[0,7,15,23,...|(1124,[0,7,15,23,...|
|[grudge, slowly, ...|positive|    0.9414|   8|(1124,[1,6,28,65,...|(1124,[1,6,28,65,...|
|[grudge, 

lDA clustering

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 6
max_iter = 10
lda = LDA(k=num_topics, 
          maxIter=max_iter, 
          featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)

In [ ]:
vocab = tf_model.vocabulary
@udf(ArrayType(StringType()))
def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]

num_top_words = 7
topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', get_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=100)

+-----+-------------------------------------------------------------+
|topic|                                                   topicWords|
+-----+-------------------------------------------------------------+
|    0|[lucille, com, try, nothing, please, congratulations, inside]|
|    1|          [attack, cease, sorrow, sit, attackshark, ooh, cry]|
|    2|       [true, lucille, color, bone, searching, someone, take]|
|    3|                   [know, need, world, true, color, susie, q]|
|    4|                  [u, fancy, maybe, free, wed, whoever, bind]|
|    5|                      [lose, donut, head, way, q, susie, say]|
+-----+-------------------------------------------------------------+



### top Negative word

In [ ]:
df_negative = df_sent_result.filter(df_sent_result.result == 'negative')
df_negative.show()

+--------------------+--------+----------+----+
|                text|  result|confidence|rank|
+--------------------+--------+----------+----+
|beyond boundary c...|positive|    0.5567|   1|
|dont need educati...|negative|    0.9207|   1|
|music john lombar...|negative|    0.8709|   2|
|run say shore tak...|negative|    0.8573|   3|
|se win zweig drei...|negative|    0.8363|   4|
|im keeper heart k...|negative|    0.8324|   5|
|engineer heavy he...|negative|    0.8229|   6|
|look around tell ...|negative|    0.8096|   7|
|dts ok go make mi...|negative|    0.8075|   8|
|cat stevens ban l...|negative|     0.806|   9|
|whoo yeah parent ...|negative|    0.8052|  10|
|gretas cedar hope...|negative|    0.8041|  11|
|want find somewhe...|negative|    0.8038|  12|
|speak word plain ...|negative|    0.7946|  13|
|lineage close dis...|negative|    0.7935|  14|
|oath must break l...|negative|    0.7799|  15|
|say goodbye troub...|negative|    0.7749|  16|
|detroit laboston ...|negative|     0.77

Perform same steps as for top positive words

# Supervised Sentiment Analysis

peronsal Notes:
Assume Repeat Calls as detractor calls and mark them as detractor calls; one with negative sentiment, we can also use range of repeat calls to define less negative, more negative sentiments

Assuming rock contains more harsh word and negative content (this is just an assumption to showcase example of supervised sentiment analysis, you can create your own logic based on business needs) create label column and use SentimentDL to train your model